In [1]:
%matplotlib inline

import sys 
import os 
import numpy as np

nb_dir = os.getcwd()
if nb_dir not in sys.path:
    sys.path.append(nb_dir)

from plotlib.loaders import *
from plotlib.plotters import *

from phdconf import config 

ylims=[0.67, 0.45, 0.57, 0.51, 0.38]

In [2]:
queries = load_queries(config.AUS_TOPIC_PATH)
broad, specific = load_query_types(queries)

In [3]:
BASE_DIR = os.path.join(os.environ["HOME"], 'phd-generated')
link_dir = os.path.join(BASE_DIR, 'anchor-text', 'dirichlet_prior')

In [4]:
index_names = ['case-topics']
qrel_paths = [config.AUS_QREL_PATH] * len(index_names)
rel_levels = [config.AUS_REL_LEVEL] * len(index_names)
display_names = ['base', 'indegree', 'sum-indegree', 'outdegree', 'sum-outdegree', 'comb', 'comb-sum']

topics = 'case-topics'

metrics = copy.copy(config.METRIC_NAMES)
del metrics['recall_100']

to = 19

In [5]:
mu = 1050
base_df = load_1d_dfs(['filtered-phrasestop'], qrel_paths, os.path.join(BASE_DIR, 'preprocessing', 'dirichlet_prior'), 'case-topics-{0}-unigram_dir_mu_{1:.2f}.run', rel_levels, mu, mu, 1)[0][0]
base_qry = load_1d_dfs(['filtered-phrasestop'], qrel_paths, os.path.join(BASE_DIR, 'preprocessing', 'dirichlet_prior'), 'case-topics-{0}-unigram_dir_mu_{1:.2f}.run', rel_levels, mu, mu, 1, per_query=True)[0][0]

In [6]:
inter = Interpolater(os.path.join(BASE_DIR, 'preprocessing', 'dirichlet_prior', 'case-topics-filtered-phrasestop-unigram_dir_mu_1050.00.run'), normalize=True)

dfs = []
for d in display_names[1:]: 
    interped_dfs = []
    for _lambda in np.arange(0, 1.0, 0.01):
        inter.interpolate(os.path.join(link_dir, 'case-topics-filtered-phrasestop-unigram_dir_mu_1050.00-linktext-{0}.run'.format(d)), _lambda, 'tmp.run')
        interped_dfs.append(load_dfs(config.AUS_QREL_PATH, config.AUS_REL_LEVEL, '', ['tmp.run'])[0])
    dfs.append(interped_dfs)

In [7]:
link_text_fig = plot_tune_1d_comp(['base', 'inlink', 'weight-inlink', 'outlink', 'weight-outlink', 'comb', 'weight-comb'], metrics, 
                    [[base_df for x in range(to+1)]] + [x[:to+1] for x in dfs], 0.00, (to)/100, 0.01, legend_x=0.995, ylims=ylims, styles=['--'])

<Figure size 1152x432 with 6 Axes>

In [8]:
link_text_fig.savefig('figures/ausnl-linktext-interp.pdf')

In [9]:
link_max = select_1d_max_with_interp(display_names[1:], dfs, 0.0, 0.01, '$\lambda$', inter, base_qry, base_df, 1050, os.path.join(link_dir, 'case-topics-filtered-phrasestop-unigram_dir_mu_1050.00-linktext-{0}.run'), config.AUS_QREL_PATH, config.AUS_REL_LEVEL, metrics=metrics).T

In [10]:
print(link_max.drop(['Unjudged@20'], axis='columns').to_latex(escape=False))

\begin{tabular}{lllllll}
\toprule
              &   &       RR &   ERR@20 &     R@20 &     NDCG &      RBP \\
\midrule
base & $\lambda$ &  1050.00 &  1050.00 &  1050.00 &  1050.00 &  1050.00 \\
              & - &   0.5686 &   0.3794 &   0.4600 &   0.4613 &   0.3113 \\
comb & $\lambda$ &     0.00 &     0.00 &     0.01 &     0.00 &     0.05 \\
              & - &   0.5686 &   0.3794 &   0.4660 &   0.4613 &   0.3164 \\
comb-sum & $\lambda$ &     0.00 &     0.00 &     0.01 &     0.00 &     0.05 \\
              & - &   0.5686 &   0.3794 &   0.4649 &   0.4613 &   0.3172 \\
indegree & $\lambda$ &     0.07 &     0.04 &     0.01 &     0.03 &     0.05 \\
              & - &   0.5758 &   0.3839 &   0.4646 &   0.4630 &   0.3168 \\
outdegree & $\lambda$ &     0.00 &     0.00 &     0.00 &     0.00 &     0.01 \\
              & - &   0.5686 &   0.3794 &   0.4600 &   0.4613 &   0.3113 \\
sum-indegree & $\lambda$ &     0.01 &     0.00 &     0.01 &     0.01 &     0.00 \\
              & - &   0.5720 &

In [11]:
inter = Interpolater(os.path.join(BASE_DIR, 'preprocessing', 'dirichlet_prior', 'case-topics-filtered-phrasestop-unigram_dir_mu_1050.00.run'), normalize=True)

names = ['iprob', 'oprob']
dfs = []
for d in names: 
    interped_dfs = []
    for _lambda in np.arange(0, 1.0, 0.01):
        inter.interpolate(os.path.join(BASE_DIR, 'links', d+'-res.txt'), _lambda, 'tmp.run')
        interped_dfs.append(load_dfs(config.AUS_QREL_PATH, config.AUS_REL_LEVEL, '', ['tmp.run'], False)[0])
    dfs.append(interped_dfs)

In [12]:
cit_max = select_1d_max_with_interp(names, dfs, 0.0, 0.01, '$\lambda$', inter, base_qry, base_df, 1050, os.path.join(BASE_DIR, 'links', '{0}-res.txt'), config.AUS_QREL_PATH, config.AUS_REL_LEVEL, metrics=metrics).T
cit_max

RR   ERR@20     R@20          NDCG      RBP Unjudged@20
base  $\lambda$  1050.00  1050.00  1050.00       1050.00  1050.00     1050.00
      -           0.5686   0.3794   0.4600        0.4613   0.3113      0.0000
iprob $\lambda$     0.06     0.06     0.01          0.00     0.06        0.94
      -           0.5768   0.3794   0.4610        0.4613   0.3129     7.70526
oprob $\lambda$     0.12     0.17     0.09          0.11     0.11        0.87
      -           0.5903   0.4077   0.4684  0.4801$^{*}$   0.3255     6.30526

In [13]:
# link_max.loc[link_comps, '$\lambda$']

## Compare to citation effectiveness

In [14]:
# max for err@20
om = copy.copy(config.METRIC_NAMES)
del om['recall_100']
del om['unjudged@20']

text_comps =['sum-indegree', 'sum-outdegree']
link_comps = ['iprob', 'oprob']
runs = ['in', 'out']

cols = om.keys()

a = pd.DataFrame()
b = pd.DataFrame()

for i in range(len(link_comps)):
    interps = [float(x) for x in link_max.loc[text_comps[i], '$\lambda$'].values]
    for j, c in zip(interps, cols):
        inter.interpolate(os.path.join(link_dir, 'case-topics-filtered-phrasestop-unigram_dir_mu_1050.00-linktext-{0}.run'.format(text_comps[i])), j, 'tmp.run')
        a[c] = load_dfs(config.AUS_QREL_PATH, config.AUS_REL_LEVEL, '', ['tmp.run'], per_query=True)[0][c]
    
    b_interps = [float(x) for x in cit_max.loc[link_comps[i], '$\lambda$'].values]
    for j, c in zip(b_interps, cols):
        inter.interpolate(os.path.join(BASE_DIR, 'links', link_comps[i]+'-res.txt'), j, 'tmp.run')
        b[c] = load_dfs(config.AUS_QREL_PATH, config.AUS_REL_LEVEL, '', ['tmp.run'], per_query=True)[0][c]
    
    qry_comp_df = a-b
    qry_comp_fig = qry_comp_df[om.keys()].rename(metrics, axis='columns').plot.box(fontsize=15, boxprops=dict(linestyle='-', linewidth=2), medianprops=dict(linestyle='-', linewidth=2), color=dict(boxes='black', whiskers='black', medians='b', caps='r'), figsize=(16, 4)).axhline(y=0, xmin=0.0, xmax=1.0, linestyle='--', linewidth=1.0, color='grey')
    qry_comp_fig.get_figure().savefig('figures/ausnl-link-{0}-qry-comp.pdf'.format(runs[i]))


/home/danlocke/.local/lib/python3.7/site-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)
/home/danlocke/.local/lib/python3.7/site-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


<Figure size 1152x288 with 1 Axes>

<Figure size 1152x288 with 1 Axes>

In [15]:
inter = Interpolater(os.path.join(BASE_DIR, 'preprocessing', 'dirichlet_prior', 'case-topics-filtered-phrasestop-unigram_dir_mu_1050.00.run'), normalize=True)

dfs = []
for d in display_names[1:]: 
    interped_dfs = []
    for _lambda in np.arange(0, 1.0, 0.01):
        inter.interpolate(os.path.join(link_dir, 'case-topics-filtered-phrasestop-unigram_dir_mu_1050.00-linktext-{0}.run'.format(d)), _lambda, 'tmp.run')
        interped_dfs.append(load_dfs(config.AUS_QREL_PATH, config.AUS_REL_LEVEL, '', ['tmp.run'], per_query=True)[0])
    dfs.append(interped_dfs)

In [16]:
tt_folds = read_folds('ausnl-folds.txt')

In [17]:
ntlm_df = pd.DataFrame(columns=metrics)

for ab, runs in zip(['indegree', 'sum-indegree', 'outdegree', 'sum-outdegree', 'comb', 'comb-sum'], dfs):
    cross = cross_validation(runs, tt_folds, metrics, base_qry)
    ntlm_df.loc[ab] = cross[0]
    
ntlm_df.loc['$R$'] = base_df.round(4)
ntlm_df = ntlm_df.rename(index={'indegree':'inlink', 'outdegree': 'outlink', 'sum-indegree': 'weighted-inlink', 'sum-outdegree': 'weighted-outlink', 'comb-sum': 'weighted-comb'})
ntlm_df = ntlm_df.reindex(['$R$', 'inlink', 'weighted-inlink', 'outlink', 'weighted-outlink', 'comb', 'weighted-comb'])
write_table('tables/ausnl-linktext', bold_max(ntlm_df).rename(columns=metrics).drop('Unjudged@20',axis='columns').to_latex(escape=False))

In [18]:
def read_count_file(path: str): 
    out = {}
    with open(path) as f:
        for line in f:
            vals = list(map(int, line.split()))
            print(len(vals))
            q = vals[0]
            out[q] = vals[1:]
            vals = vals[1:]
            split = [vals[i:i+2] for i in range(0, len(vals), 2)]
            _in = []
            _out = []
            for i, s in enumerate(split):
                if i % 2 == 0:
                    _in.append(s)
                else:
                    _out.append(s)

            out[q] = [[x[0] for x in _in], [x[1] for x in _in], [x[0] for x in _out], [x[1] for x in _out]]
    
    return out

counts = read_count_file(os.path.join(BASE_DIR, 'anchor-text', 'counts.txt'))

401
401
401
401
401
401
401
401
401
401
401
401
401
401
401
401
401
401
401
401
401
401
401
401
401
401
401
401
401
401
401
401
401
401
401
401
401
401
401
401
401
401
401
401
401
401
401
401
401
401
401
401
401
401
401
401
401
401
401
401
401
401
401
401
401
401
401
401
401
401
401
401
401
401
401
401
401
401
401
401
401
401
401
401
401
401
401
401
401
401
401
401
401
401
401


In [19]:
count_df = pd.DataFrame.from_dict(counts, orient='index', columns=['inlinks', 'pin', 'outlinks', 'pout'])

In [21]:
count_df.mean()/100

NameError: name 'count_df' is not defined

In [25]:
xs = [1, 2, 3, 4, 5, 6, 7, 8]
size = 2
xs = [xs[i:i+size] for i in range(0, len(xs), size)]
xs[1::2]

[[3, 4], [7, 8]]

In [36]:
pd.DataFrame.from_dict({k: [sum(x) for y in v for x in y] for k, v in counts.items}, orient='index')

TypeError: 'builtin_function_or_method' object is not iterable

In [30]:
pd.DataFrame.from_dict({k: [sum(x) for x in v] for k, v in counts.items()}, orient='index', columns=['inlinks', 'pin', 'outlinks', 'pout'])

inlinks  pin  outlinks  pout
45      993  224       945   235
47      700  249       725   282
55      792  189       640   154
94      917  175      1181   253
39      873  161       666   151
..      ...  ...       ...   ...
81      816  195       852   162
8       889  265       705   167
10      974  196       962   179
29     1074  229      1336   212
35      762  160       849   124

[95 rows x 4 columns]

In [69]:
def sum_relevant(qrel_path: str, res_path: str, counts): 
    qrels = {}
    with open(qrel_path) as f:
        for line in f:
            parts = line.split()
            q = int(parts[0])
            rel = qrels.get(q, [set(), set()])
            if parts[3] != '0':
                rel[0].add(parts[2])
            else:
                rel[1].add(parts[2])
            qrels[q] = rel
    
    
    rel = []
    nonrel = []
    for i in range(4):
        rel.append([])
        nonrel.append([])
    numrel = [0]*2
    with open(res_path) as f:
        for line in f:
            parts = line.split()
            q = int(parts[0])
            r = int(parts[3])
            if parts[2] in qrels[q][0]:
                numrel[0] += 1
                for i, x in enumerate(counts[q]):
                    rel[i].append(x[r])
            elif parts[2] in qrels[q][1]:
                numrel[1] += 1
                for i, x in enumerate(counts[q]):
                    nonrel[i].append(x[r])
                
                
    return np.array(rel), np.array(nonrel), numrel
                
            
    
rel_stats, non_rel_stats, proportion = sum_relevant(config.AUS_QREL_PATH, os.path.join(link_dir, 'case-topics-filtered-phrasestop-unigram_dir_mu_3000.00-linktext-indegree.run'), counts)

In [64]:
rel_stats/proportion[0]

array([ 2.44196429,  2.43154762, 15.37946429,  2.24553571])

In [66]:
non_rel_stats/proportion[1]

array([ 2.00519719,  1.9981657 , 16.23020483,  2.13115255])

In [73]:
from scipy.stats import ttest_ind

for i in range(len(rel_stats)):
    print(ttest_ind(rel_stats[i], non_rel_stats[i]))

Ttest_indResult(statistic=2.246512440986131, pvalue=0.024726353201432023)
Ttest_indResult(statistic=2.23510820411306, pvalue=0.025466059432895715)
Ttest_indResult(statistic=-1.2510111784612974, pvalue=0.21100465926214215)
Ttest_indResult(statistic=0.8772286321759563, pvalue=0.3804159382017599)


array([ 1641,  1634, 10335,  1509])